# figure 4 vlz

- kernel: r_env, R 4.1.3
- date: 2023-12-28

## load

In [ ]:
library(tidyverse)
library(logging)
library(ggpubr)
library(ggsci)
library(patchwork)
library(Seurat)

source('../scripts/r_funcs.r')

theme_set(theme_pubr())
logging::basicConfig()
options(warn = -1)

outdir <- '../figures/fig4'
create_dir(outdir)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.0     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.1     ✔ tibble    3.2.0
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Attaching SeuratObject



In [ ]:
f_pat_gp <- '../tables/patient_info_v2.tsv'
gp <- 'response'
gp_lvls$response <- c('R', 'PR')
gp_comp_map$response <- list(
  c('R-pre', 'R-post'),
  c('PR-pre', 'PR-post'),
  c('R-pre', 'PR-pre'),
  c('R-post', 'PR-post')
)
gp_comp_diff_map$response <- list(c('R', 'PR'))
gp_comp_map_pre$response <- list(c('R-pre', 'PR-pre'))
comb_order <- c('R-pre', 'R-post', 'PR-pre', 'PR-post')

In [ ]:
celltype_map <- list(
    'T_MKI67' = c('T_Prolif')
)

cell_state_map <- list(
    'cytotoxic' = c('CD8_ANXA1', 'CD8_CCL5', 'CD8_CX3CR1', 'CD8_FOS', 'CD8_GZMK', 'CD8_KLRB1'),
    'exhausted' = c('CD4_CXCL13', 'CD8_CXCL13', 'CD8_TYMS', 'T_MKI67', 'Treg_TNFRSF4'),
    'dying' = c('T_Mito'),
    'others' = c('CD4_KLRB1', 'CD8_CD74', 'CD8_IFIT1', 'T_IL7R', 'T_Ribo', 'Treg_LTB')
)
cell_state_order <- c('cytotoxic', 'exhausted', 'dying', 'others')
cell_state_color <- c('exhausted' = '#023fa5', 'cytotoxic' = 'red', 
                      'dying' = '#bb7784', 'others' = '#ff9639')

## pre-post TCR sharing: 4a-c

In [ ]:
f_share <- '../../stage4/a05_clone_share/pre_post_share/pre_post_share-per_clone_subtype.tsv'  # subtype level share
f_clone_share_sample <- '../../stage4/a05_clone_share/pre_post_share/pre_post_share-by_sample-per_clone.tsv'  # sample level share

min_cell_per_sample_subtype <- 10  # to cosistent with expand percent analysis

### 4a: vln of clone fraction in Treat, novel vs pre-exisiting

- sharing at sample level
- clone fraction (based on RNA & TCR data) in mandarad_group, post-novel & pre-post-shared types
- min_cell_per_clone = 2

In [ ]:
min_cell_per_clone <- 2
xorder <- c('R-pre_existing', 'R-post_novel', 'PR-pre_existing', 'PR-post_novel')
comp_ls <- list(
    c('R-pre_existing', 'R-post_novel'), c('PR-pre_existing', 'PR-post_novel'),
    c('R-pre_existing', 'PR-pre_existing'), c('R-post_novel', 'PR-post_novel')
)

clone_share_info <- read_tsv(f_clone_share_sample, show_col_types = F)
clone_share_info %>% dim

[1] 67283    17

In [ ]:
# stat
df_stat <- clone_share_info %>% 
    filter(!is.na(sample_post)) %>%  # all Treat clones %>%
    add_clin_info(ftsv = f_pat_gp, columns = gp, merge_by = 'patient') %>%
    mutate(xlab = case_when(
        share_type == 'pre_post_share' ~ paste0(.data[[gp]], '-pre_existing'), 
        share_type == 'post_specific' ~ paste0(.data[[gp]], '-post_novel'), 
        TRUE ~ 'others'
    )) %>%
    filter(n_cell_per_sample_clone_post >= min_cell_per_clone)  # filter clones with few cells
df_stat %>% dim
df_stat %>% write_tsv(str_glue('{outdir}/fig4a-tcr_share-post_novel-vs-pre_exist-clone_frac_new.tsv'))

2024-02-05 15:12:25 INFO::these clinial info will be added: response


[1] 4689   19

In [ ]:
# plot
p <- read_tsv(str_glue('{outdir}/fig4a-tcr_share-post_novel-vs-pre_exist-clone_frac_new.tsv'), show_col_types = F) %>%
    mutate(pat_gp = factor(.data[[gp]], gp_lvls[[gp]])) %>%
    ggviolin(x = 'xlab', y = 'clone_frac_new_post', fill = 'pat_gp', add = 'boxplot', palette = 'jco', order = xorder) +
    scale_y_continuous(trans = 'log10') +
    stat_compare_means(comparisons = comp_ls) +
    labs(x = '', y = 'Clonotype fraction at sample level\n(RNA & TCR data)', fill = 'Response') +
    theme(axis.text.x = element_text(angle = 60, hjust = 0.96))
ggsave(filename = str_glue('{outdir}/fig4a-vln_tcr_share-post_novel-vs-pre_exist-clone_frac_new.pdf'),
       plot = p, width = 4, height = 6)

### 4b-1: all Baseline vs all Treat in Baseline

- all Baseline, all Treat overlap TCRs / all Baseline TCRs, cell type level
- min_cell_per_sample_subtype = 10

In [ ]:
df <- read_tsv(f_share, show_col_types = F)

# rename cell type
for (nm in names(celltype_map)) {
    df$subtype[df$subtype %in% celltype_map[[nm]]] <- nm
}

In [ ]:
# stat
df_stat <- df %>% 
    filter(!is.na(sample_pre)) %>%  # all Baseline TCRs
    add_count(sample_pre, subtype, name = 'n_clone_per_samp_subtype') %>%
    count(patient, subtype, share_type, n_clone_per_samp_subtype, n_cell_per_sample_subtype_pre, name = 'n_clone') %>% 
    pivot_wider(names_from = 'share_type', values_from = 'n_clone', values_fill = 0) %>%
    filter(n_cell_per_sample_subtype_pre >= min_cell_per_sample_subtype) %>%
    mutate(pct = 100 * pre_post_share / n_clone_per_samp_subtype) %>%
    filter(!grepl('^NK', subtype)) %>%  # no NK cells
    add_clin_info(ftsv = f_pat_gp, columns = gp, merge_by = 'patient')
df_stat %>% write_tsv(str_glue('{outdir}/fig4b-tcr_share-pre_in_post.tsv'))
loginfo('%g recores', nrow(df_stat))

2024-02-05 15:21:59 INFO::these clinial info will be added: response
2024-02-05 15:21:59 INFO::513 recores


In [ ]:
# plot
dp <- read_tsv(str_glue('{outdir}/fig4b-tcr_share-pre_in_post.tsv'), show_col_types = F)
xorder <- dp %>%
        summarise(.by = c('subtype'), val = median(pct)) %>%
        arrange(-val) %>%
        pull('subtype')
p <- dp %>%
    cell_comp_boxplot(x = 'subtype', xorder = xorder, y = 'pct', pair_by = NULL, seed_i = 12345, size = 2,
                      pt_fill = gp, fill_order = gp_lvls[[gp]], xangle = 60, facet_by = NULL,
                      ytitle = 'Percent of shared clones in pre-treatment') +
    labs(title = str_glue('all Baseline clones vs all Treat clones,',
                          '\n>={min_cell_per_sample_subtype} cells per Baseline sample per subtype'),
         fill = 'Response') 
ggsave(filename = str_glue('{outdir}/fig4b-box_tcr_share-pre_in_post-min{min_cell_per_sample_subtype}.pdf'),
       plot = p, width = 6, height = 6)

### 4b-2: all Baseline vs all Treat in Treat

- all Baseline, all Treat overlap TCRs / all Treat TCRs, cell type level
- min_cell_per_sample_subtype = 10

In [ ]:
df <- read_tsv(f_share, show_col_types = F)
# rename cell type
for (nm in names(celltype_map)) {
    df$subtype[df$subtype %in% celltype_map[[nm]]] <- nm
}
# stat
df_stat <- df %>% 
    filter(!is.na(sample_post)) %>%  # all Treat TCRs
    add_count(sample_post, subtype, name = 'n_clone_per_samp_subtype') %>%
    count(patient, subtype, share_type, n_clone_per_samp_subtype, n_cell_per_sample_subtype_post, name = 'n_clone') %>% 
    pivot_wider(names_from = 'share_type', values_from = 'n_clone', values_fill = 0) %>% 
    filter(n_cell_per_sample_subtype_post >= min_cell_per_sample_subtype) %>%
    mutate(pct = 100 * pre_post_share / n_clone_per_samp_subtype) %>%
    filter(!grepl('^NK', subtype)) %>%  # no NK cells
    add_clin_info(ftsv = f_pat_gp, columns = gp, merge_by = 'patient')
df_stat %>% write_tsv(str_glue('{outdir}/fig4b-tcr_share-post_in_pre.tsv'))
loginfo('%g records', nrow(df_stat))

2024-02-05 15:24:26 INFO::these clinial info will be added: response
2024-02-05 15:24:26 INFO::415 records


In [ ]:
# plot
dp <- read_tsv(str_glue('{outdir}/fig4b-tcr_share-post_in_pre.tsv'), show_col_types = F)
xorder <- dp %>%
        summarise(.by = c('subtype'), val = median(pct)) %>%
        arrange(-val) %>%
        pull('subtype')
p <- dp %>%
    cell_comp_boxplot(x = 'subtype', xorder = xorder, y = 'pct', pair_by = NULL, seed_i = 12345, size = 2,
                      pt_fill = gp, fill_order = gp_lvls[[gp]], xangle = 60, facet_by = NULL,
                      ytitle = 'Percent of shared clones in post-treatment') +
    labs(title = str_glue('all Baseline clones vs all Treat clones,',
                          '\n>={min_cell_per_sample_subtype} cells per Treat sample per subtype'),
         fill = 'Response') 
ggsave(filename = str_glue('{outdir}/fig4b-box_tcr_share-post_in_pre-min{min_cell_per_sample_subtype}.pdf'),
       plot = p, width = 6, height = 6)

### 4c: share clone percent in post

In [ ]:
f_pct_info <- '../figures/fig4/fig4b-tcr_share-post_in_pre.tsv'
used_ctypes <- c(
    'CD8_FOS', 'CD8_GZMK', 'CD8_CXCL13', 
    'CD8_ANXA1', 'Treg_TNFRSF4', 'CD8_TYMS'
)

In [ ]:
p <- read_tsv(f_pct_info, show_col_types = F) %>% 
    filter(subtype %in% used_ctypes) %>%
    mutate(subtype = factor(subtype, used_ctypes)) %>%
    cell_comp_boxplot(x = gp, y = 'pct', pt_fill = gp, facet_by = 'subtype', pair_by = NULL,
                      xorder = gp_lvls[[gp]], fill_order = gp_lvls[[gp]], ncol = 3) +
    stat_compare_means(comparisons = gp_comp_diff_map[[gp]]) +
    labs(y = 'Percent of shared clones in post-treatment', fill = 'Response') +
    theme(axis.title.x = element_blank(), legend.position = 'right')
ggsave(filename = str_glue('{outdir}/fig4c-box_tcr_share-share_clone_pct_comp-by_{gp}.pdf'),
       plot = p, width = 7, height = 7)

## clone change type (pre vs post): 4d-h

In [ ]:
# cell source of 4 clone cat
f_cell_source <- '../../stage4/a05_clone_share/pre_post_share/clone_frac_in_sample/clone_cell_source-cell_state-clone_change_type.tsv'
# clone sharing intra- cell state level (with sharing_type)
f_share_type <- '../../stage4/a05_clone_share/pre_post_share/pre_post_share-per_clone-cell_state.tsv'
# merged clone frac/clone change type (sample level) & clone sharing info (no sharing_type)
f_clone_change <- '../../stage4/a05_clone_share/pre_post_share/clone_frac_in_sample/clone_pair_patient-clone_change_type-chisq.tsv'

clone_type_lvl <- c('Expanded', 'Contracted', 'Persistent')
clone_type_color_map <- c(
    'Expanded' = '#EE0000FF', 'Contracted' = '#3B4992FF', 'Persistent' = 'gray60'
)
ctype_colors <- c(
    CD8_ANXA1 = '#8C0439', CD8_CCL5 = '#A12253', CD8_CX3CR1 = '#B6416D', CD8_FOS = '#CC6088',
    CD8_GZMK = '#E17FA2', CD8_KLRB1 = '#F79EBD', CD4_CXCL13 = '#003CA5', CD8_CXCL13 = '#2F5DB1', 
    CD8_TYMS = '#5E7FBD', T_MKI67 = '#8DA1C9', Treg_TNFRSF4 = '#BDC3D6', T_Mito = '#7B7D7B',
    CD4_KLRB1 = '#DF9608', CD8_CD74 = '#E2A22E', CD8_IFIT1 = '#E5AE54', T_IL7R = '#E8BA7A', 
    T_Ribo = '#EBC6A0', Treg_LTB = '#EFD3C6'
)

### 4d: scatter show clone type

- desc: only pre-post shared clones (clone sharing defined intra cell state)

In [ ]:
xy_tick_labels <- c(0.001, 0.01, 0.1)
used_states <- c('cytotoxic', 'exhausted', 'dying', 'others')

In [ ]:
df <- read_tsv(f_clone_change, show_col_types = F) %>%
    filter(!grepl('NK', cell_state)) %>%  # no NK cells
    filter(cell_state %in% used_states) %>%
    select(-c('mandard_group', 'response_degree', 'treatment_group')) %>%
    add_clin_info(ftsv = f_pat_gp, columns = gp, merge_by = 'patient')
df <- read_tsv(f_share_type, show_col_types = F) %>% 
    filter(share_type == 'pre_post_share') %>%
    select(patient, patient_id, clonotype, cell_state, share_type) %>% 
    distinct() %>%
    merge(df, by = c('patient', 'patient_id', 'clonotype', 'cell_state'))
df %>% write_tsv(str_glue('{outdir}/figs7a-clone_change_type-clone_frac_pre_vs_post-by_state-only_share.tsv'))

loginfo('%g records', nrow(df))

2024-02-22 16:19:58 INFO::these clinial info will be added: response
2024-02-22 16:19:59 INFO::4240 records


In [ ]:
# scatter show expand type use clone fraction
p <- read_tsv(str_glue('{outdir}/figs7a-clone_change_type-clone_frac_pre_vs_post-by_state-only_share.tsv'), show_col_types = F) %>%
    mutate(clone_change_type = factor(clone_change_type, levels = clone_type_lvl),
           frow = factor(.data[[gp]], gp_lvls[[gp]]),
           fcol = factor(cell_state, used_states)) %>%
    ggscatter(x = 'clone_frac_in_sample_pre', y = 'clone_frac_in_sample_post', color = 'clone_change_type', alpha = 0.7, cor.coef = T) +
    geom_abline(slope = 1, intercept = 0) +
    facet_grid(frow ~ fcol)  +
    scale_x_continuous(breaks = xy_tick_labels) +
    scale_y_continuous(breaks = xy_tick_labels) +
    scale_color_manual(values = clone_type_color_map) +
    guides(color = guide_legend(override.aes = list(size = 4))) +
    coord_trans(x = lognp_trans(n = 0.001), y = lognp_trans(n = 0.001)) +
    labs(x = 'Clone fraction in Baseline (RNA & TCR)', y = 'Clone fraction in Treat (RNA & TCR)',
         color = 'Clone type') +
    theme(legend.position = 'right')
ggsave(filename = str_glue('{outdir}/figs7a-scatter-clone_chage_type-clone_frac_pre_vs_post-by_state-only_share.pdf'),
       plot = p, width = 15, height = 5)

### 4e: clone change type composition

- to prove cytotoxic clusters are pre low-expand to post high-expand, exhausted clusters are pre-high expand clones to post-low
- show contacted/expaned clone compositions at sample level

In [ ]:
comp_ls <- list(
    c('R-Contracted', 'R-Expanded'), 
    c('PR-Contracted', 'PR-Expanded')
)
used_states <- c('cytotoxic', 'exhausted')

In [ ]:
dp <- read_tsv(f_clone_change, show_col_types = F) %>% 
    filter(!grepl('NK', cell_state)) %>%  # no NK cells
    add_clin_info(ftsv = f_pat_gp, columns = gp, merge_by = 'patient') %>%
    add_count(patient, cell_state, name = 'n_clone_per_sample_state') %>% 
    count(patient, .data[[gp]], cell_state, clone_change_type, n_clone_per_sample_state) %>% 
    mutate(pct = 100 * n / n_clone_per_sample_state) %>%
    pivot_wider(id_cols = c('patient', gp, 'cell_state'),
                names_from = 'clone_change_type', 
                values_from = 'pct', values_fill = 0) %>%
    pivot_longer(cols = clone_type_lvl, names_to = 'clone_change_type', values_to = 'pct') %>%
    mutate(cell_state = factor(cell_state, levels = cell_state_order))
dp %>% write_tsv(str_glue('{outdir}/fig4e-clone_change_type-clone_pct-by_state.tsv'))
loginfo('%g records, %g clone type', nrow(dp), length(unique(dp$clone_change_type)))

2024-02-05 16:38:04 INFO::these clinial info will be added: response
2024-02-05 16:38:04 INFO::348 records, 3 clone type


In [ ]:
# boxplot show good vs poor clone change type proportion at sample level
p <- read_tsv(str_glue('{outdir}/fig4e-clone_change_type-clone_pct-by_state.tsv'), show_col_types = F) %>%
    filter(clone_change_type %in% c('Expanded', 'Contracted'),
           cell_state %in% used_states) %>%
    cell_comp_boxplot(x = c(gp, 'clone_change_type'),  y = 'pct', pt_fill = gp, facet_by = 'cell_state', 
                      pair_by = NULL, xorder = unlist(comp_ls), fill_order = gp_lvls[[gp]], xangle = 60) + 
    scale_y_continuous(breaks = c(1, 10, 100)) +
    stat_compare_means(comparisons = comp_ls) +
    coord_trans(y = 'lognp') + 
    labs(fill = 'Response', y = 'Clone percentage') +
    theme(legend.position = 'right', axis.title.x = element_blank())
ggsave(filename = str_glue('{outdir}/fig4e-box-clone_change_type-clone_pct-by_state.pdf'),
       plot = p, width = 6, height = 5.5)

### 4f: subtype source of interested clones

only consider pre-post paired samples; consider:
- cytotoxic-Expanded, exhausted-Contracted clones
- Contracted clones in cytotoxic clusters, Expanded clones in exhausted clusters

In [ ]:
used_cat <- c(
  'Contracted clones in exhausted clusters', 'Expanded clones in exhausted clusters',
  'Contracted clones in cytotoxic clusters', 'Expanded clones in cytotoxic clusters'
)

In [ ]:
# plot
p <- read_tsv(f_cell_source, show_col_types = F) %>%
    filter(!grepl('NK', celltype)) %>%
    filter(clone_cat %in% used_cat) %>%
    count(.data[[gp]], sample_type, clone_cat, celltype, name = 'n_cell') %>%
    mutate(response_stype = factor(paste0(.data[[gp]], '-', sample_type), comb_order),
           celltype = factor(celltype, names(ctype_colors))) %>%
    ggbarplot(x = 'n_cell', y = 'response_stype', fill = 'celltype', position = position_fill()) +
    facet_wrap(~ clone_cat) +
    scale_fill_manual(values = ctype_colors) +
    guides(fill = guide_legend(nrow = 3)) +
    labs(x = 'Cell proportion', y = '', fill = '')

ggsave(filename = str_glue('{outdir}/fig4f-bar-cell_source-cytoExpand_exhauContract.pdf'), 
       plot = p, width = 12, height = 7)

## cell percent of specific cluster in specific clone cat

- date: 2024-02-29
- desc: cell level

In [ ]:
min_cell <- 10
clone_source <- read_tsv(f_cell_source, show_col_types = F) %>% 
    filter(!grepl('NK', celltype)) 
loginfo('total %g cells, %g samples', nrow(clone_source), length(unique(clone_source$sample)))

2024-03-01 09:46:42 INFO::total 66214 cells, 58 samples


### 4g: exhausted clusters in exhausted contracted

In [ ]:
prefix <- 'fig4h-cell_pct_in_exhau_contract'
used_ctypes <- c('CD8_TYMS', 'T_MKI67')
df <- clone_source %>% 
    filter(clone_cat == 'Contracted clones in exhausted clusters') %>% 
    mutate(cellgp = if_else(celltype %in% used_ctypes, celltype, 'others')) %>% 
    count(patient, sample_type, cellgp, name = 'n_cell') %>% 
    pivot_wider(id_cols = c('patient', 'sample_type'), names_from = 'cellgp',
                values_from = 'n_cell', values_fill = 0) %>% 
    pivot_longer(cols = -c('patient', 'sample_type'), names_to = 'cellgp', values_to = 'n_cell') %>% 
    add_count(patient, sample_type, name = 'n_cell_per_sample', wt = n_cell) %>% 
    mutate(pct = 100 * n_cell / n_cell_per_sample) %>%
    add_clin_info(ftsv = f_pat_gp, columns = gp, merge_by = 'patient')
loginfo('%g records', nrow(df))
df %>% write_tsv(str_glue('{outdir}/{prefix}.tsv'))

2024-03-01 09:51:15 INFO::these clinial info will be added: response
2024-03-01 09:51:15 INFO::147 records


In [ ]:
p_ls <- lapply(X = used_ctypes, function(ctype) {
    df %>%
    filter(n_cell_per_sample >= min_cell) %>%
    filter(cellgp == ctype) %>%
    cell_comp_boxplot(x = c(gp, 'sample_type'), y = 'pct', pt_fill = gp, facet_by = NULL,
                      xorder = comb_order, fill_order = gp_lvls[[gp]], xangle = 60) +
    stat_compare_means(comparisons = gp_comp_map[[gp]]) +
    labs(y = str_glue('Cell percent of {ctype} in exhausted contracted cells'), fill = 'Response') +
    theme(legend.position = 'right')
})
ggsave(filename = str_glue('{outdir}/{prefix}-{min_cell}.pdf'), width = 4, height = 5,
       plot = wrap_plots(p_ls, ncol = 2, guides = 'collect') & theme(legend.position = 'top'))

### 4h: exhausted clusters in cytotoxic expanded

In [ ]:
prefix <- 'fig4h-cell_pct_in_cyto_expand'
used_ctypes <- c('exhausted clusters', 'T_Mito')
df <- clone_source %>% 
    filter(clone_cat == 'Expanded clones in cytotoxic clusters') %>% 
    mutate(cellgp = case_match(
        celltype,
        cell_state_map$exhausted ~ 'exhausted clusters',
        cell_state_map$dying ~ 'T_Mito',
        .default = 'others'
    )) %>% 
    count(patient, sample_type, cellgp, name = 'n_cell') %>% 
    pivot_wider(id_cols = c('patient', 'sample_type'), names_from = 'cellgp',
                values_from = 'n_cell', values_fill = 0) %>% 
    pivot_longer(cols = -c('patient', 'sample_type'), names_to = 'cellgp', values_to = 'n_cell') %>% 
    add_count(patient, sample_type, name = 'n_cell_per_sample', wt = n_cell) %>% 
    mutate(pct = 100 * n_cell / n_cell_per_sample) %>%
    add_clin_info(ftsv = f_pat_gp, columns = gp, merge_by = 'patient')
    
loginfo('%g records', nrow(df))
df %>% write_tsv(str_glue('{outdir}/{prefix}.tsv'))

2024-03-01 10:02:04 INFO::these clinial info will be added: response
2024-03-01 10:02:04 INFO::174 records


In [ ]:
p_ls <- lapply(X = used_ctypes, function(ctype) {
    df %>%
    filter(n_cell_per_sample >= min_cell) %>%
    filter(cellgp == ctype) %>%
    cell_comp_boxplot(x = c(gp, 'sample_type'), y = 'pct', pt_fill = gp, facet_by = NULL,
                      xorder = comb_order, fill_order = gp_lvls[[gp]], xangle = 60) +
    stat_compare_means(comparisons = gp_comp_map[[gp]]) +
    labs(y = str_glue('Cell percent of {ctype} in cytotoxic expanded cells'), fill = 'Response') +
    theme(legend.position = 'right')
})
ggsave(filename = str_glue('{outdir}/{prefix}-{min_cell}.pdf'), width = 4, height = 5,
       plot = wrap_plots(p_ls, ncol = 2, guides = 'collect') & theme(legend.position = 'top'))